# Overview
This notebook was created to save and track my Pytorch learning Journey. This notebook is likely to have multiple sequels as this Pytorch is quite a complex and sophisticated framework

# Fundamentals

install CUDA on your local machine if a GPU is available. Don't forget to google cuda is still not available!!!

In [2]:
import torch
torch.cuda.is_available()

True

## tensors

starting with some theory ? what is a tensor ?  
Well, first assuming the existence of a basis of $n$ vectors. then a tensor of rank $m$ is a quantity that is defined by $n^m$ components. Generally this quantity represents a phenomena that can be mathematically modeled by $m$ different vectors. Each component represents a scalar associated with a combination among the possible combinations of choosing $n$ directions (from the basis vectors) $m$ times with repetition.  
HERE IS  A GREAT [REFERENCE](https://www.youtube.com/watch?v=f5liqUk0ZTw)  
resources:  
1. [torch.tensor](https://pytorch.org/docs/stable/tensors.html)
2. [ZTM course](https://www.learnpytorch.io/00_pytorch_fundamentals/)

Nevertheless, let's get more acquainted with tensors in ***Pytorch***   
1. a scalar is a tensor of dimension $0$
2. a vector is a tensor of dimension $1$
3. a matrix is a tensor of dimension $2$
4. an image is a tensor of dimension $3$

In [3]:
# Tensor
t = torch.tensor([[[1, 2, 3],
                        [3, 6, 9],
                        [2, 4, 5]], 
                [[1, 2, 3], 
                 [1, 1, 1],
                  [2,2,2]]])

print(t.shape)
print(t.ndim)

# creating tensors is not usually done manuallym
# here is some good tensor-creation methos
t1 = torch.rand(size=(3, 4))
t2 = torch.rand(size=(2, 3, 4))

print(t1)
print(t2)


torch.Size([2, 3, 3])
3
tensor([[0.0759, 0.0145, 0.6034, 0.7342],
        [0.3004, 0.5141, 0.2457, 0.6696],
        [0.2699, 0.0861, 0.0877, 0.2000]])
tensor([[[0.8396, 0.7654, 0.2743, 0.3086],
         [0.7247, 0.2554, 0.0491, 0.5630],
         [0.3756, 0.2225, 0.5597, 0.0028]],

        [[0.1691, 0.5789, 0.5650, 0.5662],
         [0.5461, 0.4812, 0.9157, 0.7639],
         [0.5184, 0.0495, 0.8166, 0.4469]]])


In [4]:
ones1 = torch.ones(size=(3, 1))
ones2 = torch.ones(size=(3, 2))
# print(ones1, ones2, sep='\n')

# let's create some tensor and extract its characteristics
t = torch.rand(size=(2, 2, 2))
print(f"shape of tensor: {t.shape}")
print(f"data type of tensor: {t.dtype}")
print(f"the device where the tensor is stored: {t.device}")

shape of tensor: torch.Size([2, 2, 2])
data type of tensor: torch.float32
the device where the tensor is stored: cpu


Most of runtime errors would probably be cause by a mismatch of ones of these fields.

In [5]:
# as for operations with matrices, they are the same as in python, numpy
print(ones1 + ones2) # broadcasting works here
print((ones1 + 1) * (ones2 + 3.5)) # should have a (3, 2) with 9.0 for each value

tensor([[2., 2.],
        [2., 2.],
        [2., 2.]])
tensor([[9., 9.],
        [9., 9.],
        [9., 9.]])


In [13]:
# as for the matrix multiplication (not the element wise one)
# we should a built-in function where the result depends hugely on the arguments' dimensions
# in other words if the result of the multiplication is mathematically a vector, then the object will have only 1 dimension, if it is a scalar then it will be 0-dimensional
print(torch.mm(ones2.T, ones1).size()) # this will be 2 * 1 matrix: or a 2-dimensional vector
# if the data type is a problem, then we can convert it easily
t = torch.randn(size=(3, 4), dtype=torch.float16)
t = t.type(torch.float32)
print(t)

torch.Size([2, 1])
tensor([[-0.5425,  0.2715, -0.0266, -0.2744],
        [-0.3252,  0.2417, -1.5869, -0.6758],
        [ 0.3862,  1.3789, -1.4697,  0.2311]])


In [20]:
# dealing with shapes
# be mindful of views are they copied by reference
t = torch.tensor([[1, 2, 3], [2, 3, 4]], dtype=torch.float32)
t_v = t.view(1, 3, 2)

# changing a view affects the original tensor
t_v[0, 1, 1] = -1
# let's see the change reflected on 't' 
print(t, t_v, sep='\n')

tensor([[ 1.,  2.,  3.],
        [-1.,  3.,  4.]])
tensor([[[ 1.,  2.],
         [ 3., -1.],
         [ 3.,  4.]]])


In [24]:
# fed up with dimensions of value 1: squeeze() is ur best friend
squeezed_t = t_v.squeeze()
print(squeezed_t)
# with squeezing comes unsqueezing: adding another dimension of value 1: the dimension should be specified


tensor([[ 1.,  2.],
        [ 3., -1.],
        [ 3.,  4.]])
